In [1]:
import keras
from keras.layers import Dense, LSTM, Flatten, TimeDistributed, Conv2D, Dropout, MaxPooling2D
from keras import Sequential
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
import numpy as np
from PIL import Image
import glob

Using TensorFlow backend.


In [7]:
file_glob = glob.glob('Fixed_Images/*.jpg')

In [8]:
def get_time_stamp(name):
    p = 0
    img_date = name.split('_')[2:4]
    img_edt = int(name.split('_')[-1:][0].split('.')[0])
    date = '{}-{}-{} {}' .format('2020', img_date[0], img_date[1], img_edt)
    return date

In [9]:
n = file_glob[130]
get_time_stamp(n)

'2020-05-24 1210'

In [ ]:
vgg = VGG16(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

In [6]:
model = Sequential()
model.add(TimeDistributed(vgg, input_shape=(6, 224, 224, 3)))
model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2), strides=(2, 2))))
model.add(TimeDistributed(Flatten()))

In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_1 (TimeDist (None, 6, 7, 7, 512)      14714688  
_________________________________________________________________
time_distributed_2 (TimeDist (None, 6, 3, 3, 512)      0         
_________________________________________________________________
time_distributed_3 (TimeDist (None, 6, 4608)           0         
Total params: 14,714,688
Trainable params: 14,714,688
Non-trainable params: 0
_________________________________________________________________


In [8]:
#returns tensor list from 6 image slices
tensor_stack = []
def to_tensor(img_path):
    img = image.load_img(img_path, target_size=(260, 1344))
    img_arr = np.array(img)
    list_of_tensors = []
    for i in range(0,6):
        slice_show = i
        slice_size = 1344/6
        img_slice = img_arr[:224, slice_size * slice_show:slice_size * (slice_show + 1), :]
        list_of_tensors.append(np.expand_dims(img_slice, axis=0).astype('float32')/255)
    return np.vstack(list_of_tensors) 

for f in file_glob[:2]:
    tensors = to_tensor(f)
    tensor_stack.append(tensors)

In [9]:
predictions = []
for t in tensor_stack:
    t = np.array(t).reshape(1, 6, 224, 224, 3)
    preds = model.predict(t)
    predictions.append(preds)

In [10]:
len(predictions[0][0][5])

4608

In [11]:
len(predictions[0][0])

6

In [12]:
#build model
lstm = Sequential()
lstm.add(LSTM(1024, return_sequences = True, input_shape = (6, 4608)))
lstm.add(LSTM(1024))
lstm.add(Dense(1))
lstm.compile(loss='mae', optimizer='adam', metrics = ['acc'])
lstm.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 6, 1024)           23072768  
_________________________________________________________________
lstm_2 (LSTM)                (None, 1024)              8392704   
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 1025      
Total params: 31,466,497
Trainable params: 31,466,497
Non-trainable params: 0
_________________________________________________________________


In [1]:
x = predictions[0][0].reshape(1, 6, 4608)

NameError: name 'predictions' is not defined

In [ ]:
hist = lstm.fit(x, [3], epochs = 1)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/1
